In [23]:
import pandas as pd
import logging
import requests
import sys
import urllib
import bs4

from bs4 import BeautifulSoup
from collections import OrderedDict
from urllib.parse import urlencode


def get_list(words, page):

    newsData = OrderedDict()
    href = 'http://search.sina.com.cn/?%s&c=news&from=&col=&range=all&source=&country=&size=10&time=&dpc=0&a=&ps=0&pf=0&page=%s' % (words, page)
    html = requests.get(href)
    soup = BeautifulSoup(html.content, 'html.parser')
    divs = soup.findAll('div', {"class": "r-info r-info2"})
    for div in divs:
        head = div.findAll('h2')[0]
        titleinfo = head.find('a')
        title = titleinfo.get_text()
        url = titleinfo['href']
        otherinfo = head.find('span', {"class": "fgray_time"}).get_text()
        source, date, time = otherinfo.split()
        abstract = div.find('p', {"class": "content"}).get_text()
        newsData[title] = [source, date,time, abstract, url]
    return newsData



if __name__ == "__main__":
    wordsRawStr = '新冠病毒'
    words = wordsRawStr.encode('gbk')
    d = {'q': words}
    pname = urlencode(d)
    news=[]
    for page in range(1,10):
        newsData = get_list(pname, page)
        for ky in newsData:
            a = '\001'.join([ky] + newsData[ky])
            news.append(a)
            
news = pd.DataFrame(data = news)
news.to_csv("C:/Users/lenovo/Desktop/news.txt")
with open("C:/Users/lenovo/Desktop/news.txt","r",encoding="utf-8") as f:
    lines = f.readlines()
with open("C:/Users/lenovo/Desktop/news.txt","w",encoding="utf-8") as f_w:
    for line in lines:
        if ",0" in line:
            continue
        f_w.write(line)

def get_body(href):
    html = requests.get(href)
    soup = BeautifulSoup(html.content, 'html.parser')
    div = soup.find('div',{"id":"artibody"})
    content=''#以解决局部变量和全局变量的问题
    if isinstance(div,bs4.element.Tag):#应对一些中间有无法识别的空格的情况
        paras = div.find_all("p")
        content = ''
        for p in paras:
            ptext = p.get_text().strip().replace("\n","")
            content += ptext
    return content




if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)
    coronalvirus = []
    with open("C:/Users/lenovo/Desktop/news.txt",encoding="utf-8") as f:
        for line in f:
                slogan,title, source, date, abstract, href = line.strip().split('\001')
                logging.info('Scraping ' + href)
                content = get_body(href)
                b='\001'.join([slogan,title, source, date, abstract, href, content])
                coronalvirus.append(b)
news = pd.DataFrame(data = coronalvirus)
news.to_csv("C:/Users/lenovo/Desktop/coronalvirus.csv")

INFO:root:Scraping http://shenzhen.sina.com.cn/news/n/2020-04-09/detail-iirczymi5376128.shtml
INFO:root:Scraping https://k.sina.com.cn/article_5044281310_12ca99fde020018uer.html?from=news&subch=onews
INFO:root:Scraping https://mil.news.sina.com.cn/2020-04-09/doc-iircuyvh6846470.shtml
INFO:root:Scraping https://k.sina.com.cn/article_3147098824_bb94eac8020017xry.html?from=local
INFO:root:Scraping https://k.sina.com.cn/article_5044281310_12ca99fde020018uen.html?from=finance
INFO:root:Scraping https://k.sina.com.cn/article_1784473157_6a5ce64502001tjus.html?from=news&subch=onews
INFO:root:Scraping https://news.sina.com.cn/c/2020-04-09/doc-iircuyvh6845599.shtml
INFO:root:Scraping https://zx.sina.cn/e/2020-04-09/zx-iirczymi5375158.d.html
INFO:root:Scraping https://k.sina.com.cn/article_5787187353_158f1789902000zkpx.html?from=news&subch=onews
INFO:root:Scraping https://k.sina.com.cn/article_6456450127_180d59c4f02000yzzw.html?from=news&subch=onews
INFO:root:Scraping https://k.sina.com.cn/articl